# DSSM
1. Использовать информацию о качестве взаимодействия юзеров с айтемами для более репрезентативного сэмплирования. (3 балла)
2. Добавить текстовые признаки в модель. (3 балла)
3. Пробить скор бейзлайна DSSM модели 0.075. (2 балла)
4. Обернуть в сервис, который должен  критериям читаемости и воспоизводимости. (5 балла)

# Autoencoder
1. Значимо изменить архитектуру модели (2 балла)
2. Пробить скор бейзлайна DSSM модели 0.075. (2 балла)
3. Обернуть в сервис, который должен  критериям читаемости и воспоизводимости. (5 балла)

# Frameworks
1. Проэкспериментриуйте с различными моделями из выбранного фреймворка(recbole или другие рассмотренные фреймворки), выбрать лучшую. (2 балла)
2. Пробить скор бейзлайна DSSM модели 0.075. (3 балла)
3. Обернуть в сервис, который должен  критериям читаемости и воспоизводимости. (5 балла)

# RecBole

In [1]:
!pip install recbole
!pip install kmeans-pytorch
!pip install ray

import ast
import json
import logging
import os
import pickle
import time
import warnings

from collections import Counter
from logging import getLogger
from pathlib import Path
from random import randint, random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch

from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.data.interaction import Interaction
from recbole.model.general_recommender.multivae import MultiVAE
from recbole.model.sequential_recommender import Caser, GRU4Rec
from recbole.quick_start import run_recbole
from recbole.trainer import Trainer
from recbole.utils import init_logger, init_seed
from scipy.sparse import coo_matrix, hstack
from sklearn.metrics.pairwise import (
    cosine_distances,
    cosine_similarity,
    euclidean_distances,
)

## Из семинара

In [ ]:
import requests
from tqdm import tqdm

url = 'https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip'
req = requests.get(url, stream=True)

with open('kion_train.zip', "wb") as fd:
    total_size_in_bytes = int(req.headers.get('Content-Length', 0))
    progress_bar = tqdm(desc='Downloading the kion dataset...',
                        total=total_size_in_bytes,
                        unit='iB', unit_scale=True)
    for chunk in req.iter_content(chunk_size=2 ** 20):
        progress_bar.update(len(chunk))
        fd.write(chunk)



In [ ]:
!unzip kion_train.zip -x '__MACOSX/*'

In [2]:
users_df = pd.read_csv("/content/data_original/users.csv")
items_df = pd.read_csv("/content/data_original/items.csv")
interactions_df = pd.read_csv("/content/data_original/interactions.csv")

In [3]:
interactions_df.head(2)

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0


In [4]:
users_df.shape

(840197, 5)

In [5]:
interactions_df["t_dat"] = pd.to_datetime(interactions_df["last_watch_dt"], format="%Y-%m-%d")
interactions_df["timestamp"] = interactions_df.t_dat.values.astype(np.int64) // 10**9

In [6]:
df = interactions_df[['user_id', 'item_id', 'timestamp']].rename(
    columns={'user_id': 'user_id:token', 'item_id': 'item_id:token', 'timestamp': 'timestamp:float'})

In [7]:
!mkdir recbox_data

mkdir: cannot create directory ‘recbox_data’: File exists


In [8]:
df.to_csv('recbox_data/recbox_data.inter', index=False, sep='\t')

# Fit

In [9]:
parameter_dict = {
    'data_path': '',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'TIME_FIELD': 'timestamp',
    'device': 'GPU',
    'user_inter_num_interval': "[40,inf)",
    'item_inter_num_interval': "[40,inf)",
    'load_col': {'inter': ['user_id', 'item_id', 'timestamp']},
    'neg_sampling': None,
    'epochs': 10,
    'eval_args': {
        'split': {'RS': [9, 0, 1]},
        'group_by': 'user',
        'order': 'TO',
        'mode': 'full'}
}
config = Config(model='MultiVAE', dataset='recbox_data', config_dict=parameter_dict)

# init random seed
init_seed(config['seed'], config['reproducibility'])

# logger initialization
init_logger(config)
logger = getLogger()
# Create handlers
c_handler = logging.StreamHandler()
c_handler.setLevel(logging.INFO)
logger.addHandler(c_handler)

In [10]:
dataset = create_dataset(config)
logger.info(dataset)

In [11]:
train_data, valid_data, test_data = data_preparation(config, dataset)

In [12]:
%%time
model_list = ["MultiVAE", "MultiDAE"]

results = {}

for model_name in model_list:
    print(f"running {model_name}...")
    start = time.time()
    result = run_recbole(model=model_name, dataset="recbox_data", config_dict=parameter_dict)
    results[model_name] = result
    t = time.time() - start
    print(f"It took {t/60:.2f} mins")
    print(result)

running MultiVAE...


command line args [-f /root/.local/share/jupyter/runtime/kernel-608aa52a-c08a-4057-89de-ae41fe6d12a8.json] will not be used in RecBole
Max value of user's history interaction records has reached 20.9471766848816% of the total.
Evaluate   : 100%|███████████████████| 13354/13354 [02:51<00:00, 77.64it/s, GPU RAM: 0.38 G/14.75 G]


It took 3.76 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0834), ('mrr@10', 0.1671), ('ndcg@10', 0.0816), ('hit@10', 0.3466), ('precision@10', 0.0462)])}
running MultiDAE...


command line args [-f /root/.local/share/jupyter/runtime/kernel-608aa52a-c08a-4057-89de-ae41fe6d12a8.json] will not be used in RecBole
Max value of user's history interaction records has reached 20.9471766848816% of the total.
Evaluate   : 100%|███████████████████| 13354/13354 [04:00<00:00, 55.64it/s, GPU RAM: 0.38 G/14.75 G]


It took 5.73 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0837), ('mrr@10', 0.1657), ('ndcg@10', 0.0814), ('hit@10', 0.3466), ('precision@10', 0.0463)])}
CPU times: user 8min 6s, sys: 31.3 s, total: 8min 38s
Wall time: 9min 29s


# Inference

In [13]:
results

{'MultiVAE': {'best_valid_score': -inf,
  'valid_score_bigger': True,
  'best_valid_result': None,
  'test_result': OrderedDict([('recall@10', 0.0834),
               ('mrr@10', 0.1671),
               ('ndcg@10', 0.0816),
               ('hit@10', 0.3466),
               ('precision@10', 0.0462)])},
 'MultiDAE': {'best_valid_score': -inf,
  'valid_score_bigger': True,
  'best_valid_result': None,
  'test_result': OrderedDict([('recall@10', 0.0837),
               ('mrr@10', 0.1657),
               ('ndcg@10', 0.0814),
               ('hit@10', 0.3466),
               ('precision@10', 0.0463)])}}

In [33]:
config = Config(model='MultiVAE', dataset='recbox_data', config_dict=parameter_dict)

model = MultiVAE(config, dataset)

loaded_model = torch.load("/content/saved/MultiVAE-Dec-20-2023_13-15-44.pth")
model.load_state_dict(loaded_model["state_dict"])

command line args [-f /root/.local/share/jupyter/runtime/kernel-608aa52a-c08a-4057-89de-ae41fe6d12a8.json] will not be used in RecBole
Max value of user's history interaction records has reached 23.254401942926535% of the total.


<All keys matched successfully>

In [37]:
model = model.to('cuda:0')

In [46]:
def recommend_items(user_id, dataset, model, k_recos):
    if (user_id in dataset.field2token_id[dataset.uid_field] and user_id != "[PAD]"):
        model.eval()
        user = np.isin(dataset[dataset.uid_field].numpy(), dataset.token2id(dataset.uid_field, [user_id]))
        dataset_user = dataset[user]
        predict = model.full_sort_predict(dataset_user).view(-1, test_data.dataset.item_num)
        predict[:, 0] = -np.inf
        item_indices = torch.topk(predict, k_recos).indices[0].to_list()
        recos = dataset.id2token(dataset.iid_field, [item_indices]).to_list()
    else:
        recos = []
    return recos

In [47]:
recos_data = []
users = dataset.field2token_id[dataset.uid_field]

counter = 0
for user_id in users:
    counter+=1
    recos_for_user = recommend_items(user_id, dataset, model,10)
    if recos_for_user:
        recos_data.extend([(user_id, item_id) for item_id in recos_for_user])
    if counter % 100 == 0:
        print(f"Counter value: {counter}")

recos_df = pd.DataFrame(recos_data, columns=["user_id", "item_id"])

AttributeError: ignored

In [9]:
with open('multiae.json', 'wb') as file:
    json.dump(recos_df, file)
